## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions
0,0,Kalmunai,LK,2022-01-30 22:04:17,7.4167,81.8167,76.33,88,50,5.06,scattered clouds
1,1,Punta Arenas,CL,2022-01-30 22:04:17,-53.1500,-70.9167,51.91,50,75,23.02,broken clouds
2,2,Hilo,US,2022-01-30 22:02:19,19.7297,-155.0900,78.12,58,83,5.01,broken clouds
3,3,Longyearbyen,SJ,2022-01-30 22:04:18,78.2186,15.6401,8.44,61,0,20.71,clear sky
4,4,Albany,US,2022-01-30 22:01:50,42.6001,-73.9662,19.15,41,11,4.00,few clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 75
What is the maximum temperature you would like for your vacation? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions
0,0,Kalmunai,LK,2022-01-30 22:04:17,7.4167,81.8167,76.33,88,50,5.06,scattered clouds
2,2,Hilo,US,2022-01-30 22:02:19,19.7297,-155.0900,78.12,58,83,5.01,broken clouds
7,7,Atuona,PF,2022-01-30 22:02:49,-9.8000,-139.0333,78.08,77,11,13.69,light rain
9,9,Rikitea,PF,2022-01-30 22:01:23,-23.1203,-134.9692,76.55,69,100,4.16,overcast clouds
13,13,Vaini,TO,2022-01-30 22:04:21,-21.2000,-175.2000,80.15,77,100,12.71,light rain
18,18,Arraial Do Cabo,BR,2022-01-30 22:04:23,-22.9661,-42.0278,76.86,88,75,10.36,light rain
23,23,Mahebourg,MU,2022-01-30 22:04:25,-20.4081,57.7000,79.09,94,75,11.50,light intensity shower rain
24,24,Kapaa,US,2022-01-30 22:01:38,22.0752,-159.3190,80.58,75,12,4.00,few clouds
27,27,Palembang,ID,2022-01-30 22:01:42,-2.9167,104.7458,77.09,94,20,2.30,few clouds
31,31,Tamandare,BR,2022-01-30 22:04:28,-8.7597,-35.1047,79.36,80,98,8.52,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID               183
City                  183
Country               183
Date                  183
Lat                   183
Lng                   183
Max Temp              183
Humidity              183
Cloudiness            183
Wind Speed            183
Current Conditions    183
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Conditions", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
0,Kalmunai,LK,76.33,scattered clouds,7.4167,81.8167,
2,Hilo,US,78.12,broken clouds,19.7297,-155.0900,
7,Atuona,PF,78.08,light rain,-9.8000,-139.0333,
9,Rikitea,PF,76.55,overcast clouds,-23.1203,-134.9692,
13,Vaini,TO,80.15,light rain,-21.2000,-175.2000,
18,Arraial Do Cabo,BR,76.86,light rain,-22.9661,-42.0278,
23,Mahebourg,MU,79.09,light intensity shower rain,-20.4081,57.7000,
24,Kapaa,US,80.58,few clouds,22.0752,-159.3190,
27,Palembang,ID,77.09,few clouds,-2.9167,104.7458,
31,Tamandare,BR,79.36,overcast clouds,-8.7597,-35.1047,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]  
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

hotel_df.head(10) 

,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
0,Kalmunai,LK,76.33,scattered clouds,7.4167,81.8167,VS Villa
2,Hilo,US,78.12,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Atuona,PF,78.08,light rain,-9.8000,-139.0333,Villa Enata
9,Rikitea,PF,76.55,overcast clouds,-23.1203,-134.9692,People ThankYou
13,Vaini,TO,80.15,light rain,-21.2000,-175.2000,Keleti Beach Resort
18,Arraial Do Cabo,BR,76.86,light rain,-22.9661,-42.0278,Pousada Porto Praia
23,Mahebourg,MU,79.09,light intensity shower rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
24,Kapaa,US,80.58,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
27,Palembang,ID,77.09,few clouds,-2.9167,104.7458,ASTON Palembang Hotel & Conference Center
31,Tamandare,BR,79.36,overcast clouds,-8.7597,-35.1047,Pousada 100 Passos


In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                  183
Country               183
Max Temp              183
Current Conditions    183
Lat                   183
Lng                   183
Hotel Name            183
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Conditions</dt><dd>{Current Conditions}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.7)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=6)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))